In [1]:
from birdclef.utils import get_spark

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 15:27:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/11 15:27:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
df_sample = spark.read.csv(
    "gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2024/sample_submission.csv",
    header=True,
)
df_sample.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- asbfly: string (nullable = true)
 |-- ashdro1: string (nullable = true)
 |-- ashpri1: string (nullable = true)
 |-- ashwoo2: string (nullable = true)
 |-- asikoe2: string (nullable = true)
 |-- asiope1: string (nullable = true)
 |-- aspfly1: string (nullable = true)
 |-- aspswi1: string (nullable = true)
 |-- barfly1: string (nullable = true)
 |-- barswa: string (nullable = true)
 |-- bcnher: string (nullable = true)
 |-- bkcbul1: string (nullable = true)
 |-- bkrfla1: string (nullable = true)
 |-- bkskit1: string (nullable = true)
 |-- bkwsti: string (nullable = true)
 |-- bladro1: string (nullable = true)
 |-- blaeag1: string (nullable = true)
 |-- blakit1: string (nullable = true)
 |-- blhori1: string (nullable = true)
 |-- blnmon1: string (nullable = true)
 |-- blrwar1: string (nullable = true)
 |-- bncwoo3: string (nullable = true)
 |-- brakit1: string (nullable = true)
 |-- brasta1: string (nullable = true)
 |-- brcful1: string (nul

In [4]:
df = spark.read.csv("/mnt/data/tmp/submission.csv", header=True)
df.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- asbfly: string (nullable = true)
 |-- ashdro1: string (nullable = true)
 |-- ashpri1: string (nullable = true)
 |-- ashwoo2: string (nullable = true)
 |-- asikoe2: string (nullable = true)
 |-- asiope1: string (nullable = true)
 |-- aspfly1: string (nullable = true)
 |-- aspswi1: string (nullable = true)
 |-- barfly1: string (nullable = true)
 |-- barswa: string (nullable = true)
 |-- bcnher: string (nullable = true)
 |-- bkcbul1: string (nullable = true)
 |-- bkrfla1: string (nullable = true)
 |-- bkskit1: string (nullable = true)
 |-- bkwsti: string (nullable = true)
 |-- bladro1: string (nullable = true)
 |-- blaeag1: string (nullable = true)
 |-- blakit1: string (nullable = true)
 |-- blhori1: string (nullable = true)
 |-- blnmon1: string (nullable = true)
 |-- blrwar1: string (nullable = true)
 |-- bncwoo3: string (nullable = true)
 |-- brakit1: string (nullable = true)
 |-- brasta1: string (nullable = true)
 |-- brcful1: string (nul

In [7]:
df_sample.columns == df.columns

True

In [8]:
df.show()

24/06/11 15:31:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+-------+------+-------+-------+-------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+------+-------+------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+------+-------+-------+------+-------+------+-------+-------+-------+------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+--